In [3]:
import pandas as pd
import numpy as np
import gspread
import os
import sqlite3
import requests
from pandas.io.json import json_normalize

In [4]:
print(pd.__version__)
print(np.__version__)

0.25.3
1.18.1


#### Data directory
- all data directory assign here

In [7]:
data_path = '../data'

In [66]:
# CSV File
data_boston_calendar = pd.read_csv(os.path.join(data_path, 'boston', 'calendar.csv'))
data_boston_listings = pd.read_csv(os.path.join(data_path, 'boston', 'listings.csv'))
data_boston_reviews = pd.read_csv(os.path.join(data_path, 'boston', 'reviews.csv'))

# SQLITE
data_pitchfork = os.path.join(data_path, 'database.sqlite')

# DB
data_chinook = os.path.join(data_path, 'chinook.db')

# JSON File
data_json_file = pd.read_json(os.path.join(data_path, 'tweet_data.json'), lines=True)

# EXCEL File
data_xls_file = pd.read_excel(os.path.join(data_path, 'file_1000.xls'))

#### Review shape of file

In [62]:
print('data boston calendar : ',data_boston_calendar.shape)
print('data boston listing : ',data_boston_listings.shape)
print('data bostong reviews : ',data_boston_reviews.shape)
print('data tweet json : ', data_json_file.shape)
print('data excel : ', data_xls_file.shape)

data boston calendar :  (1308890, 4)
data boston listing :  (3585, 95)
data bostong reviews :  (68275, 6)
data tweet json :  (1000, 30)
data excel :  (1000, 8)


#### FLAT FILE - CSV

In [17]:
data_boston_calendar.head(5)

,listing_id,date,available,price
0,12147973,2017-09-05,f,NaN
1,12147973,2017-09-04,f,NaN
2,12147973,2017-09-03,f,NaN
3,12147973,2017-09-02,f,NaN
4,12147973,2017-09-01,f,NaN


In [18]:
data_boston_listings.head(2)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",...,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN
1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...",...,9.0,f,NaN,NaN,t,moderate,f,f,1,1.3


In [20]:
data_boston_reviews.head(2)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,1178162,4724140,2013-05-21,4298113,Olivier,My stay at islam's place was really cool! Good...
1,1178162,4869189,2013-05-29,6452964,Charlotte,Great location for both airport and city - gre...


#### EXCEL File

In [64]:
data_xls_file.head(10)

,Unnamed: 0,First Name,Last Name,Gender,Country,Age,Date,Id
0,1,Dulce,Abril,Female,United States,32,15/10/2017,1562
1,2,Mara,Hashimoto,Female,Great Britain,25,16/08/2016,1582
2,3,Philip,Gent,Male,France,36,21/05/2015,2587
3,4,Kathleen,Hanner,Female,United States,25,15/10/2017,3549
4,5,Nereida,Magwood,Female,United States,58,16/08/2016,2468
5,6,Gaston,Brumm,Male,United States,24,21/05/2015,2554
6,7,Etta,Hurn,Female,Great Britain,56,15/10/2017,3598
7,8,Earlean,Melgar,Female,United States,27,16/08/2016,2456
8,9,Vincenza,Weiland,Female,United States,40,21/05/2015,6548
9,10,Fallon,Winward,Female,Great Britain,28,16/08/2016,5486


#### SQLite File

In [21]:
con = sqlite3.connect(data_pitchfork)

In [27]:
df = pd.read_sql_query("""SELECT 
                                name
                            FROM 
                                sqlite_master 
                            WHERE 
                                type ='table' AND 
                                name NOT LIKE 'sqlite_%';
                            """, con)

In [30]:
df.head()

,name
0,reviews
1,artists
2,genres
3,labels
4,years


In [31]:
df_artist = pd.read_sql_query("select * from artists limit 5", con)

In [32]:
df_artist

,reviewid,artist
0,22703,massive attack
1,22721,krallice
2,22659,uranium club
3,22661,kleenex
4,22661,liliput


In [68]:
conn_chinook = sqlite3.connect(data_chinook)

In [70]:
df_chinook = pd.read_sql_query("""SELECT 
                                name
                            FROM 
                                sqlite_master 
                            WHERE 
                                type ='table' AND 
                                name NOT LIKE 'sqlite_%';
                            """, conn_chinook)

In [71]:
df_chinook

,name
0,albums
1,artists
2,customers
3,employees
4,genres
5,invoices
6,invoice_items
7,media_types
8,playlists
9,playlist_track


#### JSON API File

In [33]:
api_url = "https://reqres.in/api/users?page=2"

In [34]:
response = requests.get(api_url)

In [35]:
data_json = response.json()

In [36]:
data_json

{'page': 2,
 'per_page': 6,
 'total': 12,
 'total_pages': 2,
 'data': [{'id': 7,
   'email': 'michael.lawson@reqres.in',
   'first_name': 'Michael',
   'last_name': 'Lawson',
   'avatar': 'https://s3.amazonaws.com/uifaces/faces/twitter/follettkyle/128.jpg'},
  {'id': 8,
   'email': 'lindsay.ferguson@reqres.in',
   'first_name': 'Lindsay',
   'last_name': 'Ferguson',
   'avatar': 'https://s3.amazonaws.com/uifaces/faces/twitter/araa3185/128.jpg'},
  {'id': 9,
   'email': 'tobias.funke@reqres.in',
   'first_name': 'Tobias',
   'last_name': 'Funke',
   'avatar': 'https://s3.amazonaws.com/uifaces/faces/twitter/vivekprvr/128.jpg'},
  {'id': 10,
   'email': 'byron.fields@reqres.in',
   'first_name': 'Byron',
   'last_name': 'Fields',
   'avatar': 'https://s3.amazonaws.com/uifaces/faces/twitter/russoedu/128.jpg'},
  {'id': 11,
   'email': 'george.edwards@reqres.in',
   'first_name': 'George',
   'last_name': 'Edwards',
   'avatar': 'https://s3.amazonaws.com/uifaces/faces/twitter/mrmoiree/128.j

In [43]:
print(type(data_json))
print(type(response))

<class 'dict'>
<class 'requests.models.Response'>


In [37]:
employee = pd.DataFrame(data_json["data"])

In [38]:
employee

,id,email,first_name,last_name,avatar
0,7,michael.lawson@reqres.in,Michael,Lawson,https://s3.amazonaws.com/uifaces/faces/twitter...
1,8,lindsay.ferguson@reqres.in,Lindsay,Ferguson,https://s3.amazonaws.com/uifaces/faces/twitter...
2,9,tobias.funke@reqres.in,Tobias,Funke,https://s3.amazonaws.com/uifaces/faces/twitter...
3,10,byron.fields@reqres.in,Byron,Fields,https://s3.amazonaws.com/uifaces/faces/twitter...
4,11,george.edwards@reqres.in,George,Edwards,https://s3.amazonaws.com/uifaces/faces/twitter...
5,12,rachel.howell@reqres.in,Rachel,Howell,https://s3.amazonaws.com/uifaces/faces/twitter...


#### Another example : NESTED JSON FILE

In [53]:
data_json_file.head(5)

,contributors,truncated,text,is_quote_status,in_reply_to_status_id,in_reply_to_user_id,id,favorite_count,entities,retweeted,...,in_reply_to_user_id_str,lang,created_at,in_reply_to_status_id_str,place,quoted_status_id,quoted_status,possibly_sensitive,quoted_status_id_str,extended_entities
0,NaN,False,RT @IM3Ooredoo: Apa rencanamu utk long weekend...,False,NaN,NaN,857842893309960192,0,"{'symbols': [], 'user_mentions': [{'indices': ...",False,...,NaN,in,2017-04-28 06:24:18+00:00,NaN,None,NaN,NaN,NaN,NaN,NaN
1,NaN,False,😂😂 RT @arievrahman: *menunggu tanggapan @IM3Oo...,True,NaN,NaN,857842750812704772,0,"{'symbols': [], 'user_mentions': [{'indices': ...",False,...,NaN,in,2017-04-28 06:23:44+00:00,NaN,None,8.578357e+17,"{'contributors': None, 'truncated': False, 'te...",0.0,8.578357e+17,NaN
2,NaN,False,@IndosatCare @IM3Ooredoo mana suaranya? 😂 #tel...,True,NaN,548904824.0,857842733259542528,0,"{'symbols': [], 'user_mentions': [{'indices': ...",False,...,548904824.0,in,2017-04-28 06:23:40+00:00,NaN,None,8.578357e+17,"{'contributors': None, 'truncated': False, 'te...",0.0,8.578357e+17,NaN
3,NaN,False,RT @arievrahman: *menunggu tanggapan @IM3Oored...,True,NaN,NaN,857842605349941248,0,"{'symbols': [], 'user_mentions': [{'indices': ...",False,...,NaN,in,2017-04-28 06:23:09+00:00,NaN,None,8.578357e+17,NaN,0.0,8.578357e+17,NaN
4,NaN,False,RT @arievrahman: *menunggu tanggapan @IM3Oored...,True,NaN,NaN,857842458993897473,0,"{'symbols': [], 'user_mentions': [{'indices': ...",False,...,NaN,in,2017-04-28 06:22:34+00:00,NaN,None,8.578357e+17,NaN,0.0,8.578357e+17,NaN


In [49]:
entities = json_normalize(data_json_file["entities"], sep="_")

In [50]:
list(entities)

['symbols', 'user_mentions', 'hashtags', 'urls', 'media']

In [51]:
entities.head(5)

,symbols,user_mentions,hashtags,urls,media
0,[],"[{'indices': [3, 14], 'screen_name': 'IM3Oored...",[],[],NaN
1,[],"[{'indices': [6, 18], 'screen_name': 'arievrah...",[],"[{'indices': [53, 76], 'url': 'https://t.co/jn...",NaN
2,[],"[{'indices': [0, 12], 'screen_name': 'IndosatC...","[{'indices': [42, 52], 'text': 'telkomsel'}, {...","[{'indices': [71, 94], 'url': 'https://t.co/tJ...",NaN
3,[],"[{'indices': [3, 15], 'screen_name': 'arievrah...",[],"[{'indices': [50, 73], 'url': 'https://t.co/54...",NaN
4,[],"[{'indices': [3, 15], 'screen_name': 'arievrah...",[],"[{'indices': [50, 73], 'url': 'https://t.co/54...",NaN


In [55]:
entities_nested = json_normalize(data_json_file["entities"], sep="_",
                                record_path="user_mentions",
                                errors='ignore'
                                   )

In [56]:
entities_nested

,indices,screen_name,id,name,id_str
0,"[3, 14]",IM3Ooredoo,61379637,IM3 Ooredoo,61379637
1,"[6, 18]",arievrahman,79921163,muhammad arif rahman,79921163
2,"[40, 51]",IM3Ooredoo,61379637,IM3 Ooredoo,61379637
3,"[0, 12]",IndosatCare,548904824,Indosat Ooredoo Care,548904824
4,"[13, 24]",IM3Ooredoo,61379637,IM3 Ooredoo,61379637
...,...,...,...,...,...
1747,"[0, 11]",IM3Ooredoo,61379637,IM3 Ooredoo,61379637
1748,"[0, 12]",IndosatCare,548904824,Indosat Ooredoo Care,548904824
1749,"[13, 24]",IM3Ooredoo,61379637,IM3 Ooredoo,61379637
1750,"[0, 12]",IndosatCare,548904824,Indosat Ooredoo Care,548904824


### Why we need to understand this ? 